In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [2]:
train_datagen =ImageDataGenerator(
                                rescale=1./255,
                                rotation_range=30,
                                shear_range=0.3,
                                zoom_range=0.3,
                                width_shift_range=0.4,
                                height_shift_range=0.4,
                                horizontal_flip=True,
                                fill_mode="nearest")

In [3]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
                                                    './train/',
                                                    color_mode="grayscale",
                                                    target_size=(48,48),
                                                    batch_size=64,
                                                    class_mode="categorical",
                                                    shuffle=True)

Found 24177 images belonging to 5 classes.


In [5]:
validation_generator = validation_datagen.flow_from_directory(  
                                                    './validation/',
                                                    color_mode="grayscale",
                                                    target_size=(48,48),
                                                    batch_size=64,
                                                    class_mode="categorical",
                                                    shuffle=True)

Found 3006 images belonging to 5 classes.


In [6]:
model = Sequential()

In [7]:
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(48,48,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(48,48,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

In [8]:
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

In [9]:
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

In [10]:
model.add(Flatten())
model.add(Dense(128,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())

In [11]:
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())

In [12]:
model.add(Dense(32,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())

In [13]:
model.add(Dense(5, kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

In [15]:
checkpoint = ModelCheckpoint("Emotion_Model.h5",
                            monitor='val_loss',
                            mode='min',
                            save_best_only=True,
                            verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                         min_delta=0,
                         patience=4,
                         verbose=1,
                         restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                             factor=0.2,
                             patience=3,
                             verbose=1,
                             min_delta=0.0001)

In [16]:
callback = [earlystop, checkpoint, reduce_lr]

In [17]:
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

In [18]:
history = model.fit_generator(train_generator, epochs=25, steps_per_epoch=24177//64, callbacks=callback, validation_data=validation_generator, validation_steps=3006//64)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
377/377 [==============================] - ETA: 0s - loss: 1.6111 - accuracy: 0.2864
Epoch 00001: val_loss improved from inf to 1.68636, saving model to Emotion_Model.h5
377/377 [==============================] - 137s 363ms/step - loss: 1.6111 - accuracy: 0.2864 - val_loss: 1.6864 - val_accuracy: 0.2897 - lr: 0.0010
Epoch 2/25
377/377 [==============================] - ETA: 0s - loss: 1.5408 - accuracy: 0.3059
Epoch 00002: val_loss improved from 1.68636 to 1.54702, saving model to Emotion_Model.h5
377/377 [==============================] - 136s 361ms/step - loss: 1.5408 - accuracy: 0.3059 - val_loss: 1.5470 - val_accuracy: 0.3217 - lr: 0.0010
Epoch 3/25
377/377 [==============================] - ETA: 0s - loss: 1.5033 - accuracy: 0.3305
Epoch 00003: val_loss improved from 1.54702 to 1.52978, saving model to Emotion_Model.h5
377/377 [==============================] - 135s 358ms/step - loss: 1.5033 - a